In [145]:
import pandas as pd
import numpy as np
import re

In [146]:
df_critics = pd.read_csv('critics_final.csv', index_col='id')
df_films = pd.read_csv('films_selenium_raw.csv', index_col='id')
df_reviews = pd.read_csv('reviews_final.csv', index_col='id')
df_directors = pd.read_csv('directors_selenium_raw.csv', index_col='id')
df_directions = pd.read_csv('directions_final.csv', index_col='id')
df_genres = pd.read_csv('genres_final.csv', index_col='id')
df_countries = pd.read_csv('countries-psql.csv', index_col='id', keep_default_na=False)
df_continents = pd.read_csv('continents-psql.csv', index_col='id', keep_default_na=False)

In [147]:
df_genres[df_genres['film_id'] == 0]

,film_id,genre
id,,
0,0,Drama
1401,0,Mystery
1402,0,Thriller


In [148]:
df_merged = (df_reviews
 .merge(df_critics, left_on='critic_id', right_index=True)
 .merge(df_films, left_on='film_id', right_index=True)
 .merge(df_directions, left_on='film_id', right_on='film_id')
 .merge(df_directors, left_on='director_id', right_index=True))

df_merged[df_merged['full_name'] == 'Alyssa Rosenberg'].sort_values('rank')

,critic_id,film_id,rank,full_name,nationality_code,company,title,year,rating,duration,director_id,name,year_of_birth,nationality
730,139,71,0,Alyssa Rosenberg,US,The Washington Post,Zodiac,2007,7.7,PT157M,74,David Fincher,1962.0,USA
1309,139,213,1,Alyssa Rosenberg,US,The Washington Post,"Crouching Tiger, Hidden Dragon",2000,7.9,PT120M,86,Ang Lee,1954.0,Taiwan
308,139,18,2,Alyssa Rosenberg,US,The Washington Post,Zero Dark Thirty,2012,7.4,PT157M,19,Kathryn Bigelow,1951.0,USA
221,139,12,3,Alyssa Rosenberg,US,The Washington Post,Spirited Away,2001,8.6,PT125M,12,Hayao Miyazaki,1941.0,Japan
765,139,77,4,Alyssa Rosenberg,US,The Washington Post,2046,2004,7.5,PT129M,1,Kar-Wai Wong,1956.0,China
909,139,97,5,Alyssa Rosenberg,US,The Washington Post,Moonrise Kingdom,2012,7.8,PT94M,16,Wes Anderson,1969.0,USA
1614,139,366,6,Alyssa Rosenberg,US,The Washington Post,Beasts of the Southern Wild,2012,7.3,PT93M,299,Benh Zeitlin,1982.0,USA
490,139,39,7,Alyssa Rosenberg,US,The Washington Post,Boyhood,2014,7.9,PT165M,39,Richard Linklater,1960.0,USA
1432,139,267,8,Alyssa Rosenberg,US,The Washington Post,Short Term 12,2013,8.0,PT96M,231,Destin Daniel Cretton,1978.0,USA
1850,139,525,9,Alyssa Rosenberg,US,The Washington Post,Monster,2003,7.3,PT109M,397,Patty Jenkins,1971.0,USA


In [149]:
df_merged[df_merged['film_id'] == 393]

,critic_id,film_id,rank,full_name,nationality_code,company,title,year,rating,duration,director_id,name,year_of_birth,nationality
1673,80,393,3,Manuel Kalmanovitz,CO,Revista Semana,Sumas y Restas,2004,6.8,PT108M,448,Victor Gaviria,1955.0,Colombia


In [150]:
df_genres[df_genres['genre'] == 'Short'].merge(df_films, left_on='film_id', right_index=True)

,film_id,genre,title,year,rating,duration
id,,,,,,
485,198,Short,World of Tomorrow,2015,8.2,PT17M
764,318,Short,Capitalism: Child Labor,2007,6.6,PT14M
765,319,Short,Psalm III: 'Night of the Meek',2002,7.2,PT24M
766,320,Short,Daylight Moon,2002,8.9,PT13M
767,322,Short,The Fourth Watch,2000,6.8,PT9M
1066,454,Short,Lifeline,2002,7.6,PT5M
1071,456,Short,Instructions for a Light and Sound Machine,2005,6.8,PT17M
1090,466,Short,Michelangelo Eye to Eye,2004,7.0,PT15M
1401,321,Short,A Commuter’s Life (What a Life!),2014,NaN,PT19M


In [204]:
df_directors = pd.read_csv('directors_selenium_raw.csv')

In [205]:
df_directors[75:85]

,id,name,year_of_birth,nationality
75,76,Shane Carruth,1972.0,USA
76,77,Peter Watkins,1935.0,UK
77,79,Carlos Reygadas,1971.0,Mexico
78,80,David Lynch,1946.0,USA
79,81,Alfonso Cuarón,1961.0,Mexico
80,82,Fernando Meirelles,1955.0,Brazil
81,83,Kátia Lund,1966.0,Brazil
82,84,Brad Bird,1957.0,USA
83,85,Guy Bolongaro,NaN,NaN
84,86,Ang Lee,1954.0,Taiwan


In [206]:
df_merged = df_directors.merge(df_countries, left_on='nationality', right_on='name', how='left')

In [207]:
df_merged[df_merged['code'].isnull()]['nationality'].unique()

array(['China', 'USA', nan, 'UK', 'South Korea', 'Taiwan', 'Yugoslavia',
       'Palestine'], dtype=object)

In [208]:
countries = {
    'UK':'GB', 
    'USA':'US', 
    'Palestine':'PS', 
    'Taiwan':'TW', 
    'China':'CN',
    'Yugoslavia':'MK',
    'South Korea':'KR'
}

In [209]:
df_merged.loc[df_merged['code'].isnull(), 'code'] = df_merged['nationality']

In [210]:
df_merged['code'].replace(countries, inplace=True)

In [212]:
df_merged[75:85]

,id,name_x,year_of_birth,nationality,name_y,code,continent_code
id,,,,,,,
75,76,Shane Carruth,1972.0,USA,NaN,US,NaN
76,77,Peter Watkins,1935.0,UK,NaN,GB,NaN
77,79,Carlos Reygadas,1971.0,Mexico,Mexico,MX,NA
78,80,David Lynch,1946.0,USA,NaN,US,NaN
79,81,Alfonso Cuarón,1961.0,Mexico,Mexico,MX,NA
80,82,Fernando Meirelles,1955.0,Brazil,Brazil,BR,SA
81,83,Kátia Lund,1966.0,Brazil,Brazil,BR,SA
82,84,Brad Bird,1957.0,USA,NaN,US,NaN
83,85,Guy Bolongaro,NaN,NaN,NaN,NaN,NaN


In [213]:
df_merged[['id', 'name_x', 'year_of_birth', 'code']].to_csv('directors_final.csv', index=False)

In [214]:
df_films = pd.read_csv('films_selenium_raw.csv')

In [215]:
df_films.head()

,id,title,year,rating,duration
0,0,Mulholland Drive,2001,8.0,PT147M
1,1,In the Mood for Love,2000,8.1,PT98M
2,2,The Tree of Life,2011,6.8,PT139M
3,3,Yi Yi: A One and a Two,2000,8.1,PT173M
4,4,Goodbye to Language,2014,5.9,PT70M


In [217]:
df_films['duration'] = df_films['duration'].str.extract('PT(\d*)M')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [218]:
df_films.head()

,id,title,year,rating,duration
0,0,Mulholland Drive,2001,8.0,147
1,1,In the Mood for Love,2000,8.1,98
2,2,The Tree of Life,2011,6.8,139
3,3,Yi Yi: A One and a Two,2000,8.1,173
4,4,Goodbye to Language,2014,5.9,70


In [219]:
df_films.to_csv('films_final.csv', index=False)